## Radiation map

We would like a model to have as inputs the irradiance of the sensors and as outputs the irradiance on a rectangular grid. Let us split the sensors in two groups: inner sensors and outer sensors. Given an initial set, would it be possible to decide the location of new outer sensors to maximize the predicibility of the inner sensors (based on irradiance data)?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

%matplotlib inline

In [2]:
DATA_PATH = '/home/SHARED/SOLAR/data/'

In [3]:
# read minute data and location info
df   = pd.read_pickle(DATA_PATH + 'oahu_min_final.pkl')
info = pd.read_pickle(DATA_PATH + 'info.pkl')

In [4]:
df_long = (df.stack()
             .reset_index('Datetime')
             .join(info[['Longitude', 'Latitude']])
             .rename(columns={0: 'GHI'})
             .dropna())

In [5]:
def get_grid(lonlat, decimals=3):
    
    prec = 10**decimals
    max_min = lonlat.agg([np.max, np.min])

    lat = np.arange(int(math.floor(max_min.loc['amin',  'Latitude']*prec)), 
                    int(math.ceil( max_min.loc['amax',  'Latitude']*prec))+1)/prec

    lon = np.arange(int(math.floor(max_min.loc['amin', 'Longitude']*prec)), 
                    int(math.ceil( max_min.loc['amax', 'Longitude']*prec))+1)/prec

    xx, yy = np.meshgrid(lon, lat)
    return pd.DataFrame({'lon': xx.ravel(), 'lat': yy.ravel()})

In [6]:
df_dict = {}
for datetime, df in df_long.groupby('Datetime'):
    X_train = df[['Longitude', 'Latitude']]
    y_train = df['GHI']
    
    X_test = get_grid(df[['Longitude', 'Latitude']])
    
    gpr = GaussianProcessRegressor(kernel=RBF(10, (1e-3, 1e-3)), n_restarts_optimizer=10)
    gpr.fit(X_train, y_train)
    #X_test['mean'], X_test['std'] = gpr.predict(X_test, return_std=True)
    X_test['GHI'] = gpr.predict(X_test)
    df_dict[datetime] = X_test.set_index(['lon', 'lat'])
    break

In [7]:
df_final = pd.concat(df_dict).unstack(level=['lon', 'lat']).sort_index(axis=1)

In [8]:
df_final.columns = df_final.columns.droplevel(0)

In [9]:
df_final.shape

(1, 108)

In [10]:
def plot_GP_heat(X_test):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    df1 = X_test.pivot_table(values='mean', columns='lon', index='lat')
    sns.heatmap(data=df1,
                xticklabels=df1.columns.values.round(3),
                yticklabels=df1.index.values.round(3),
                ax=ax[0])

    df2 = X_test.pivot_table(values='std', columns='lon', index='lat')
    sns.heatmap(data=df2,
                xticklabels=df2.columns.values.round(3),
                yticklabels=df2.index.values.round(3),
                ax=ax[1]);

In [11]:
def plot_GP_scatter(X_test):
    fig, ax = plt.subplots()
    ax.scatter(X_test['lon'], X_test['lat'], c=X_test['std'], s=300)
    ax.scatter(X_train['Longitude'], X_train['Latitude'], c='black')
    ax.set_xlim((X_test['lon'].min()-1e-3, X_test['lon'].max()+1e-3))
    ax.set_ylim((X_test['lat'].min()-1e-3, X_test['lat'].max()+1e-3));

In [12]:
#df_final.to_pickle('{}/oahu_GP.pkl'.format(DATA_PATH))